In [1]:
import warnings
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer, transpile, IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import pandas as pd
import math

from qiskit.quantum_info import Statevector

from scipy.stats import chi2_contingency, ttest_ind, levene

import unittest
import hypothesis.strategies as st
from hypothesis import given, settings

pi = np.pi

In [18]:
def measure_z(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_x(circuit, qubitIndexes):
    cBitIndex = 0
    for index in qubitIndexes:
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

def measure_y(circuit, qubit_indexes):
    cBitIndex = 0
    for index in qubit_indexes:
        circuit.sdg(index)
        circuit.h(index)
        circuit.measure(index, cBitIndex)
        cBitIndex+=1
    return circuit

In [169]:
# Completed but more testing required
def assertEntangled(backend,qc,qubits_to_assert,measurements_to_make,alpha = 0.05):
    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")
    
    zQuantumCircuit = measure_z(qc, qubits_to_assert)
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    q1=[]
    q2=[]

    classicalQubitIndex = 1
    for qubit in qubits_to_assert:
        for measurement in zMemory:
            if (measurement[2-classicalQubitIndex] == '0'):
                if(qubit==qubits_to_assert[0]):
                    q1.append(measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])
            else:
                if(qubit==qubits_to_assert[0]):
                    q1.append(measurement[2-classicalQubitIndex])
                else:
                    q2.append(measurement[2-classicalQubitIndex])                   
        classicalQubitIndex+=1

    measDict = dict.fromkeys(['qubit1','qubit2'])
    measDict = {'qubit1': q1,'qubit2':q2}
    measDf = pd.DataFrame.from_dict(measDict)
    ct = pd.crosstab(measDf.qubit1,measDf.qubit2)

    chiVal, pVal, dOfFreedom, exp = chi2_contingency(ct)
    print("chi square value: ",chiVal,"p value: ",pVal,"expected values: ",exp)
    if(pVal>alpha):
        raise(AssertionError("states are not entangled"))
    else:
        print("states are entangled")
        
backend = Aer.get_backend('aer_simulator')
qc3 = QuantumCircuit(2)
qc3.x(1)
# qc3.x(1)
# qc3.x(0)
qc3.h(0)
qc3.cnot(0,1)
print(qc3)
assertEntangled(backend,qc3,[0,1],200,0.05)

     ┌───┐     
q_0: ┤ H ├──■──
     ├───┤┌─┴─┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘
chi square value:  196.01643280573975 p value:  1.545888708011713e-44 expected values:  [[49.955 47.045]
 [53.045 49.955]]
states are entangled


In [86]:
# Completed but more testing required
## assert that qubits are equal
def assertEqual(backend, quantumCircuit, qubits_to_assert, measurements_to_make, alpha):
    ## needs to make at least 2 measurements, one for x axis, one for y axis
    ## realistically we need more for any statistical significance
    if (measurements_to_make < 2):
        raise ValueError("Must make at least 2 measurements")

    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (quantumCircuit.num_clbits == 0):
        quantumCircuit.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")

    ## divide measurements to make by 3 as we need to run measurements twice, one for x and one for y
    measurements_to_make = measurements_to_make // 3

    ## copy the circit and set measurement to y axis
    yQuantumCircuit = measure_y(quantumCircuit.copy(), qubits_to_assert)
    yQuantumCircuit2 = measure_y(quantumCircuit.copy(), qubits_to_assert)

    ## measure the x axis
    xQuantumCircuit = measure_x(quantumCircuit.copy(), qubits_to_assert)
    xQuantumCircuit2 = measure_x(quantumCircuit.copy(), qubits_to_assert)

    ## measure the z axis
    zQuantumCircuit = measure_z(quantumCircuit.copy(), qubits_to_assert)
    zQuantumCircuit2 = measure_z(quantumCircuit, qubits_to_assert)

    ## get y axis results
    yJob = execute(yQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    yMemory = yJob.result().get_memory()
    yCounts = yJob.result().get_counts()

    yJob2 = execute(yQuantumCircuit2, backend, shots=measurements_to_make, memory=True)
    yCounts2 = yJob.result().get_counts()

    ## get x axis results
    xJob = execute(xQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    xMemory = xJob.result().get_memory()
    xCounts = xJob.result().get_counts()

    xJob = execute(xQuantumCircuit2, backend, shots=measurements_to_make, memory=True)
    xCounts2 = xJob.result().get_counts()

    ## get z axis results
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()

    zJob2 = execute(zQuantumCircuit2, backend, shots=measurements_to_make, memory=True)
    zCounts2 = zJob.result().get_counts()


    resDf = pd.DataFrame(columns=['0','1','+','i','-','-i'])
    resDf2 = pd.DataFrame(columns=['0','1','+','i','-','-i'])

    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount, plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0,0,0
        for experiment in xCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                plus_amount += xCounts[experiment]
            else:
                minus_amount += xCounts[experiment]
        for experiment in yCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                i_amount += yCounts[experiment]
            else:
                minus_i_amount += yCounts[experiment]
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount,
              '+':plus_amount, 'i':i_amount,
              '-':minus_amount,'-i':minus_i_amount}

        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount, plus_amount, i_amount, minus_amount, minus_i_amount = 0,0,0,0,0,0
        for experiment in xCounts2:
            if (experiment[2-classical_qubit_index] == '0'):
                plus_amount += xCounts2[experiment]
            else:
                minus_amount += xCounts2[experiment]
        for experiment in yCounts2:
            if (experiment[2-classical_qubit_index] == '0'):
                i_amount += yCounts2[experiment]
            else:
                minus_i_amount += yCounts2[experiment]
        for experiment in zCounts2:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts2[experiment]
            else:
                one_amount += zCounts2[experiment]
        df = {'0':zero_amount, '1':one_amount,
              '+':plus_amount, 'i':i_amount,
              '-':minus_amount,'-i':minus_i_amount}

        resDf2 = resDf2.append(df, ignore_index = True)
        classical_qubit_index+=1

    resDf2['+'] = resDf2['+'].astype(int)
    resDf2['i'] = resDf2['i'].astype(int)
    resDf2['-'] = resDf2['-'].astype(int)
    resDf2['-i'] = resDf2['-i'].astype(int)
    resDf2['0'] = resDf2['0'].astype(int)
    resDf2['1'] = resDf2['1'].astype(int)

    ## convert the columns to a strict numerical type
    resDf['+'] = resDf['+'].astype(int)
    resDf['i'] = resDf['i'].astype(int)
    resDf['-'] = resDf['-'].astype(int)
    resDf['-i'] = resDf['-i'].astype(int)
    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)
    print("First Measurement:")
    print(resDf.astype(str))
    print("Second Measurement:")
    print(resDf2.astype(str))
    q1Vals = []
    q2Vals = []
    q1Vals.extend([resDf.at[0,'1'],resDf.at[0,'-'],resDf.at[0,'-i'], resDf2.at[0,'1'],resDf2.at[0,'-'],resDf.at[0,'-i'] ])
    print(q1Vals)
    q2Vals.extend([resDf.at[1,'1'],resDf.at[1,'-'],resDf.at[1,'-i'], resDf2.at[1,'1'],resDf2.at[1,'-'],resDf2.at[1,'-i']  ])
    print(q2Vals)

    # arr = resDf.to_numpy()
    # q1Vals = arr[0, 0:] # Stores measurements across all axes for the 1st qubit
    # print("Measurements for qubit1: ", q1Vals)
    # q2Vals = arr[1, 0:] # Stores measurements across all axes for the 2nd qubit
    # print("Measurements for qubit2: ", q2Vals)

    # res = levene(q1Vals, q2Vals, center= 'mean')
    # print(res)

    tTest, pValue = ttest_ind(q1Vals, q2Vals, alternative = 'two-sided') # Apply t test
    print("stat: ",tTest, "pValue: ", pValue)
    if pValue > alpha:
        print("The two qubits are equal (fail to reject null hypothesis) ")
    else:
        print("There is a significant difference between the two qubits (reject null hypothesis)")


    # ct = pd.crosstab(resDf['0'],resDf['1'])

    # print(ct)
    # chiVal, pVal, dOfFreedom, exp = chi2_contingency(ct)
    # print("exp: ",exp,"chiVal: ",chiVal,"pvalue: ",pVal)

qc = QuantumCircuit(2)
backend = Aer.get_backend('aer_simulator')
# qc.initialize([0, 1/np.sqrt(2), -1.j/np.sqrt(2), 0], qc.qubits)
qc.h(0)
qc.cnot(0,1)



# qc.p(0.5*2*math.pi/100, 1)
# qc.h(1)
# qc.p(10*2*math.pi/100, 0)
# qc.p(20*2*math.pi/100, 1)
assertEqual(backend, qc, [0,0], 300000, 0.05)
assertEqual(backend, qc, [0,0], 300000, 0.05)

First Measurement:
       0      1      +      i      -     -i
0  50091  49909  50106  50252  49894  49748
1  50091  49909  49835  50038  50165  49962
Second Measurement:
       0      1      +      i      -     -i
0  50091  49909  50016  50252  49984  49748
1  50091  49909  49908  50038  50092  49962
[49909, 49894, 49748, 49909, 49984, 49748]
[49909, 50165, 49962, 49909, 50092, 49962]
stat:  -2.3135825138986226 pValue:  0.043242814244979624
There is a significant difference between the two qubits (reject null hypothesis)
First Measurement:
       0      1      +      i      -     -i
0  49719  50281  50239  50151  49761  49849
1  49719  50281  50079  49973  49921  50027
Second Measurement:
       0      1      +      i      -     -i
0  49719  50281  49774  50151  50226  49849
1  49719  50281  49907  49973  50093  50027
[50281, 49761, 49849, 50281, 50226, 49849]
[50281, 49921, 50027, 50281, 50093, 50027]
stat:  -0.5462476331193891 pValue:  0.5968712475094778
The two qubits are equal (fa

In [ ]:
# Completed but more testing required
def assertEntangled(backend,qc,qubits_to_assert,measurements_to_make,alpha = 0.05):
    # makes sure qubits_to_assert is a list
    if (not isinstance(qubits_to_assert, list)):
        qubits_to_assert = [qubits_to_assert]

    ## classical register must be of same length as amount of qubits to assert
    ## if there is no classical register add them according to length of qubit list
    if (qc.num_clbits == 0):
        qc.add_register(ClassicalRegister(len(qubits_to_assert)))
    elif (len(qubits_to_assert) != 2):
        raise ValueError("QuantumCircuit classical register must be of length 2")
    
    zQuantumCircuit = measure_z(qc, qubits_to_assert)
    zJob = execute(zQuantumCircuit, backend, shots=measurements_to_make, memory=True)
    zMemory = zJob.result().get_memory()
    zCounts = zJob.result().get_counts()

    # zDf = pd.DataFrame(columns=['q0', 'q1'])
    # for row in zCounts:
    #     zDf = zDf.append({'q0':row[0], 'q1':row[1]}, ignore_index=True)
    # print(zDf.astype(str))
    resDf = pd.DataFrame(columns=['0','1'])
    classical_qubit_index = 1
    for qubit in qubits_to_assert:
        zero_amount, one_amount = 0,0
        for experiment in zCounts:
            if (experiment[2-classical_qubit_index] == '0'):
                zero_amount += zCounts[experiment]
            else:
                one_amount += zCounts[experiment]
        df = {'0':zero_amount, '1':one_amount}
        resDf = resDf.append(df, ignore_index = True)
        classical_qubit_index+=1

    resDf['0'] = resDf['0'].astype(int)
    resDf['1'] = resDf['1'].astype(int)
    print(resDf.astype(str))


    ct = pd.crosstab(resDf['0'],resDf['1'],margins=True)
    print(ct)

    # obs = np.array([ct.iloc[0][0:3].values,ct.iloc[1][0:3].values])
    # print(obs)

    # print(resDf2.astype(str))
    chiVal, pVal, dOfFreedom, exp = chi2_contingency(ct)
    print("chi square value: ",chiVal,"p value: ",pVal,"expected values: ",exp)
    if(pVal>alpha):
        raise(AssertionError("states are not entangled"))
    else:
        print("states are entangled")


backend = Aer.get_backend('aer_simulator')
qc3 = QuantumCircuit(2)
# qc3.x(1)
# qc3.x(1)
qc3.h(0)
# qc3.x(0)
# qc3.h(0)
qc3.cnot(0,1)
# qc3.h(1)
# qc3.h(0)
assertEntangled(backend,qc3,[0,0],100000,0.05)

In [93]:
df = pd.read_csv("https://raw.githubusercontent.com/researchpy/Data-sets/master/blood_pressure.csv")
df.info()

print(df['bp_after'][df['sex'] == 'Male'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patient    120 non-null    int64 
 1   sex        120 non-null    object
 2   agegrp     120 non-null    object
 3   bp_before  120 non-null    int64 
 4   bp_after   120 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 4.8+ KB
0     153
1     170
2     168
3     142
4     141
5     147
6     133
7     141
8     131
9     125
10    164
11    159
12    135
13    159
14    153
15    126
16    162
17    134
18    136
19    150
20    168
21    155
22    136
23    132
24    160
25    160
26    136
27    183
28    152
29    162
30    151
31    139
32    175
33    184
34    151
35    171
36    157
37    159
38    140
39    174
40    167
41    158
42    168
43    159
44    153
45    164
46    169
47    148
48    185
49    163
50    146
51    160
52    175
53    163
54    185
55    146
56    176
57 

In [80]:
# Complete needs to be checked

# Assertion to check if expected probability of viewing a particular qubitstate is equal to its actual probaility
# Qubitchoice is an optional argument that if passed will only compare the expected probabilty with the probability of observing
# that particular qubit (first or second) in the desired qubitState for the qc 
def assertProbability(qc, qubitState :str, expectedProbability, qubitChoice=None):
    sv = Statevector.from_label("00") # Creates a statevector with states 00
    evl = sv.evolve(qc) # Passes the qc into the statevector in order to evolve 
    # Performs a check to observe if qubitChoice has been passed or not 
    if(qubitChoice!=None):
        probs = evl.probabilities_dict([qubitChoice]) # If passed we will get the probabilities for that particular qubit
    else:
        probs = evl.probabilities_dict()
    probsRound = {key: round(values,2) for key,values in probs.items()} # rounds off the probabilities in the dictionary

    # Loops over the prob dictionary with rounded values 
    for key,value in probsRound.items():
        if(key==qubitState):
            if(value==expectedProbability):
                print("Expected Probability present")
                return True
            else:
                raise(AssertionError("Probability not present"))
    raise(AssertionError("Probability not present or Desired state has no probability"))
            

qc = QuantumCircuit(2)
qc.h(0)
assertProbability(qc,"0",1,1)
print("------")
assertProbability(qc,"00",0.5)

Expected Probability present
------
Expected Probability present


True